In [9]:
import os
import pandas as pd
from tqdm import tqdm

#  Adjust this path to where class folders are:
base_dir = r"C:\Users\chand\garbage cleaner\images\Garbage classification"

# Class-to-eco-score mapping
eco_score_map = {
    "cardboard": 7,
    "paper": 9,
    "glass": 8,
    "metal": 6,
    "plastic": 3,
    "trash": 1
}

#  Collect data
image_data = []

for label in eco_score_map:
    class_folder = os.path.join(base_dir, label)
    if os.path.isdir(class_folder):
        for file in tqdm(os.listdir(class_folder), desc=f" Scanning '{label}'"):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(class_folder, file)
                image_data.append({
                    "filename": image_path,
                    "label": label,
                    "eco_score": eco_score_map[label]
                })
    else:
        print(f"Folder not found: {class_folder}")

# Save CSV
df = pd.DataFrame(image_data)
csv_path = r"C:\Users\chand\garbage cleaner\eco_labels.csv"
df.to_csv(csv_path, index=False)

print(f" Saved CSV with {len(df)} rows at: {csv_path}")


 Scanning 'trash': 100%|██████████████████████████████████████████████████████████| 137/137 [00:00<00:00, 68415.25it/s]

 Saved CSV with 2527 rows at: C:\Users\chand\garbage cleaner\eco_labels.csv


In [10]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, applications, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [16]:
import pandas as pd

df = pd.read_csv("C:/Users/chand/garbage cleaner/eco_labels.csv")
print(df.columns)


Index(['filename', 'label', 'eco_score'], dtype='object')


In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CSV
df = pd.read_csv("C:/Users/chand/garbage cleaner/eco_labels.csv")

# Split dataset
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.33, stratify=temp_df['label'], random_state=42)

# Image generators
img_size = (224, 224)
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255)

def get_generator(df, shuffle=True):
    return datagen.flow_from_dataframe(
        dataframe=df,
        x_col='filename',         # <- ✅ Fix here
        y_col='label',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=shuffle
    )


train_gen = get_generator(train_df)
val_gen = get_generator(val_df, shuffle=False)
test_gen = get_generator(test_df, shuffle=False)

# Eco-score arrays
y_train_reg = train_df['eco_score'].values
y_val_reg = val_df['eco_score'].values
y_test_reg = test_df['eco_score'].values



Found 1768 validated image filenames belonging to 6 classes.
Found 508 validated image filenames belonging to 6 classes.
Found 251 validated image filenames belonging to 6 classes.


In [10]:
import numpy as np
import math
from tensorflow.keras.utils import Sequence

class DualOutputSequence(Sequence):
    def __init__(self, image_gen, eco_scores):
        self.image_gen = image_gen
        self.eco_scores = np.array(eco_scores)
        self.batch_size = image_gen.batch_size

    def __len__(self):
        return math.ceil(len(self.image_gen.filenames) / self.batch_size)

    def __getitem__(self, idx):
        # Get image batch and classification labels batch from ImageDataGenerator
        x_batch, y_batch_class = self.image_gen[idx]

        # Get regression targets batch and reshape to (batch_size, 1)
        y_batch_reg = self.eco_scores[idx * self.batch_size : (idx + 1) * self.batch_size]
        y_batch_reg = y_batch_reg.reshape(-1, 1)

        # Return inputs and outputs as a dict with output layer names matching model outputs
        return x_batch, {"class_output": y_batch_class, "eco_output": y_batch_reg}

    def on_epoch_end(self):
        self.image_gen.on_epoch_end()
train_gen_dual = DualOutputSequence(train_gen, y_train_reg)
val_gen_dual = DualOutputSequence(val_gen, y_val_reg)


In [11]:
from tensorflow.keras import layers, models, applications, optimizers, callbacks

base_model = applications.ConvNeXtTiny(
    include_top=False,
    input_shape=(224, 224, 3),
    weights='imagenet'
)
base_model.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)

# Outputs
class_output = layers.Dense(6, activation='softmax', name="class_output")(x)
eco_output = layers.Dense(1, activation='linear', name="eco_output")(x)

model = models.Model(inputs=base_model.input, outputs=[class_output, eco_output])

model.compile(
    optimizer='adam',
    loss={
        'class_output': 'sparse_categorical_crossentropy',
        'eco_output': 'mse'
    },
    metrics={
        'class_output': 'accuracy',
        'eco_output': 'mae'
    }
)
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_prestem_normal… │ (None, 224, 224, 3)       │               0 │ input_layer_5[0][0]        │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stem            │ (None, 56, 56, 96)        │           4,896 │ convnext_tiny_prestem_nor… │
│ (Sequential)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │           4,800 │ convnext_tiny_stem[0][0]   │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │             192 │ convnext_tiny_stage_0_blo… │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 384)       │          37,248 │ convnext_tiny_stage_0_blo… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 384)       │               0 │ convnext_tiny_stage_0_blo… │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │          36,960 │ convnext_tiny_stage_0_blo… │
│ (Dense)                       │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │              96 │ convnext_tiny_stage_0_blo… │
│ (LayerScale)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │               0 │ convnext_tiny_stage_0_blo… │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_18 (Add)                  │ (None, 56, 56, 96)        │               0 │ convnext_tiny_stem[0][0],  │
│                               │                           │                 │ convnext_tiny_stage_0_blo… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ convnext_tiny_stage_0_block_… │ (None, 56, 56, 96)        │           4,800 │ add_18[0][0]               │
│ (Conv2D)                      │                           │               

 Total params: 28,018,791 (106.88 MB)

 Trainable params: 198,663 (776.03 KB)

 Non-trainable params: 27,820,128 (106.13 MB)

In [12]:
from tensorflow.keras import callbacks

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = callbacks.ModelCheckpoint(
    "dual_best_model.h5",
    monitor='val_class_output_accuracy',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    train_gen_dual,
    validation_data=val_gen_dual,
    epochs=5,
    callbacks=[checkpoint]
)


C:\Users\chand\miniconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - class_output_accuracy: 0.2026 - class_output_loss: 1.9178 - eco_output_loss: 12.1009 - eco_output_mae: 2.7247 - loss: 14.0213
Epoch 1: val_class_output_accuracy improved from -inf to 0.23622, saving model to dual_best_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 643s 11s/step - class_output_accuracy: 0.2030 - class_output_loss: 1.9157 - eco_output_loss: 12.0275 - eco_output_mae: 2.7166 - loss: 13.9459 - val_class_output_accuracy: 0.2362 - val_class_output_loss: 1.6833 - val_eco_output_loss: 6.0718 - val_eco_output_mae: 2.0551 - val_loss: 7.7600
Epoch 2/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - class_output_accuracy: 0.2658 - class_output_loss: 1.6812 - eco_output_loss: 5.9827 - eco_output_mae: 2.0136 - loss: 7.6636
Epoch 2: val_class_output_accuracy improved from 0.23622 to 0.35039, saving model to dual_best_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 742s 12s/step - class_output_accuracy: 0.2665 - class_output_loss: 1.6808 - eco_output_loss: 5.9846 - eco_output_mae: 2.0140 - loss: 7.6649 - val_class_output_accuracy: 0.3504 - val_class_output_loss: 1.6457 - val_eco_output_loss: 6.0178 - val_eco_output_mae: 2.0099 - val_loss: 7.6666
Epoch 3/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step - class_output_accuracy: 0.3425 - class_output_loss: 1.6163 - eco_output_loss: 6.0754 - eco_output_mae: 2.0479 - loss: 7.6667 
Epoch 3: val_class_output_accuracy improved from 0.35039 to 0.35827, saving model to dual_best_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 1626s 30s/step - class_output_accuracy: 0.3427 - class_output_loss: 1.6160 - eco_output_loss: 6.0762 - eco_output_mae: 2.0480 - loss: 7.6676 - val_class_output_accuracy: 0.3583 - val_class_output_loss: 1.5840 - val_eco_output_loss: 6.0758 - val_eco_output_mae: 2.0095 - val_loss: 7.6637
Epoch 4/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/step - class_output_accuracy: 0.3631 - class_output_loss: 1.5570 - eco_output_loss: 6.1363 - eco_output_mae: 2.0157 - loss: 7.6698 
Epoch 4: val_class_output_accuracy improved from 0.35827 to 0.39764, saving model to dual_best_model.h5


56/56 ━━━━━━━━━━━━━━━━━━━━ 1417s 25s/step - class_output_accuracy: 0.3629 - class_output_loss: 1.5569 - eco_output_loss: 6.1353 - eco_output_mae: 2.0158 - loss: 7.6690 - val_class_output_accuracy: 0.3976 - val_class_output_loss: 1.5587 - val_eco_output_loss: 6.1612 - val_eco_output_mae: 2.1175 - val_loss: 7.7255
Epoch 5/5
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - class_output_accuracy: 0.3797 - class_output_loss: 1.5224 - eco_output_loss: 6.4844 - eco_output_mae: 2.1318 - loss: 8.0183 
Epoch 5: val_class_output_accuracy did not improve from 0.39764
56/56 ━━━━━━━━━━━━━━━━━━━━ 1295s 22s/step - class_output_accuracy: 0.3799 - class_output_loss: 1.5223 - eco_output_loss: 6.4797 - eco_output_mae: 2.1307 - loss: 8.0135 - val_class_output_accuracy: 0.3740 - val_class_output_loss: 1.5044 - val_eco_output_loss: 6.0268 - val_eco_output_mae: 1.9880 - val_loss: 7.5317


In [13]:
model.save("final_garbage_model.h5")
